In [1]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import scrabble_assistant as sa
import dictionary_preprocessing as dp


%matplotlib inline

# Выкидываем неподходящие слова из словаря

In [2]:
dp.drop_incorrect_words()

# Разобьем исходый словарь на подсловари

In [3]:
dp.make_sub_dictionaries()

# Считываем датафрейм

In [4]:
rus_words_data = pd.read_csv('dictionary.txt', header=None, names=['word'])
rus_words_data

,word
0,абажур
1,абаз
2,абазин
3,абазинка
4,абака
...,...
31060,ящер
31061,ящерица
31062,ящик
31063,ящур


In [5]:
rus_words_data['value'] = rus_words_data.word.apply(lambda x: sa.calculate_word_value(x))
rus_words_data

,word,value
0,абажур,15
1,абаз,10
2,абазин,12
3,абазинка,15
4,абака,8
...,...,...
31060,ящер,16
31061,ящерица,28
31062,ящик,16
31063,ящур,18


# Отранжируем слова по цене (без учета бонусных полей на доске).

In [6]:
rus_words_data.sort_values(by='value', ascending=False)

,word,value
30094,шлифовальщица,56
30050,шифровальщица,56
30405,экзаменующаяся,54
30102,шлихтовальщица,53
2220,брошюровщица,53
...,...,...
6446,ер,3
15702,ом,3
839,ар,3
283,ад,3


# Вы не видели никаких предупреждений. Блокнот идеален.
<img alt="Медики научились стирать память | Новые Известия" class="n3VNCb" src="https://newizv.ru/attachments/317af9c89fc354109c47c029ed16c501e5db3bb5/store/fill/1200/675/b8d575d01c39e48984e1c38e7749fb2cb856c4c95c1aec1038be76212e09/b8d575d01c39e48984e1c38e7749fb2cb856c4c95c1aec1038be76212e09.jpg" data-noaft="1" jsname="HiaYvf" jsaction="load:XAeZkd;" style="width: 434px; height: 244.125px; margin: 0px;">

# Найдем все слова, которые можем составить с заданным набором букв.

In [7]:
rus_words_data['available'] = rus_words_data.word.apply(lambda x: dp.is_word_available(Counter('абвгдеё'), x))
rus_words_data.query('available == True')

,word,value,available
283,ад,3,True
1402,бег,7,True
2613,веда,6,True
6389,еда,4,True


# Из букв 'абвгдеё' можно составить вышеперчисленные слова.

# Тест функций get_marked_rows и get_regex_patterns:

In [8]:
test_board = [
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', 'т', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', 'о', '', '', '', '', ''],
        ['', '', '', 'п', 'о', 'c', 'е', 'л', 'о', 'к', '', '', '', '', ''],
        ['', '', '', 'а', '', 'а', '', '', '', '', '', 'р', '', '', ''],
        ['', '', '', 'п', '', 'д', 'о', 'м', '', 'я', '', 'е', '', '', ''],
        ['', '', '', 'а', '', '', '', 'а', 'з', 'б', 'у', 'к', 'а', '', ''],
        ['', '', '', '', '', 'с', 'о', 'м', '', 'л', '', 'а', '', '', ''],
        ['', '', '', 'я', 'м', 'а', '', 'а', '', 'о', '', '', '', '', ''],
        ['', '', '', '', '', 'л', '', '', '', 'к', 'и', 'т', '', '', ''],
        ['', '', '', '', 'с', 'о', 'л', 'ь', '', 'о', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    ]

In [9]:
test_row_after_sharping = sa.get_marked_rows(test_board)[5]
test_row_after_sharping

['', '', '', 'а', '#', 'а', '#', '#', '#', '#', '', 'р', '', '', '']

In [10]:
patterns = sa.get_regex_patterns(test_row_after_sharping)
patterns

['^([а-я]?[а-я]?[а-я]?а)$', '^([а-я]?р[а-я]?[а-я]?[а-я]?)$']

In [11]:
rus_words_data.word.str.extract('^([а-я]{,3}а)$', flags=re.UNICODE).dropna()

,0
358,айва
857,арба
896,арка
934,арфа
970,арча
...,...
30982,яйла
31001,яма
31020,ярка
31051,яхта


In [12]:
for pattern in patterns:
    print(rus_words_data.word.str.extract(pattern, flags=re.UNICODE).dropna())

          0
358    айва
857    арба
896    арка
934    арфа
970    арча
...     ...
30982  яйла
31001   яма
31020  ярка
31051  яхта
31059  яшма

[192 rows x 1 columns]
           0
839       ар
840     араб
846     арак
853     арат
857     арба
...      ...
30939  юрист
30945   юрок
30946   юрта
31019    ярд
31020   ярка

[290 rows x 1 columns]
